## Midterm

This is due on Thurs, 10/25/18

1. Consider the polynomial $p(x) = (x-2)^9 = x^9 - 18x^8 + 144x^7 - 672x^6 + 2016x^5 - 4032x^4 + 5376x^3 - 4608x^2 + 2304x - 512$

  a. Plot $p(x)$ for $x=1.920,\,1.921,\,1.922,\ldots,2.080$ evaluating $p$ via its coefficients $1,\,,-18,\,144,\ldots$

  b. Plot the same plot again, now evaluating $p$ via the expression $(x-2)^9$.

  c. Explain the difference.
  
  *(The numpy method linspace will be useful for this)*

2\. From the Halide Video, what are 4 ways to traverse a 2d array?

3\. Using the animations below ([source](https://www.youtube.com/watch?v=3uiEyEKji0M)), explain what the benefits and pitfalls of each approach. Green squares indicate that a value is being read; red indicates a value is being written. Your answers should be longer in length (give more detail) than just two words.

  a. <img src="Halide1.gif" alt="Halide" style="width: 70%"/>
  
  b. <img src="Halide2.gif" alt="Halide" style="width: 70%"/>
  
  c. <img src="Halide3.gif" alt="Halide" style="width: 70%"/>

4\. Prove that if $A = Q B Q^T$ for some orthnogonal matrix $Q$, the $A$ and $B$ have the same singular values.

5\. Consider a particular species of wildflower in which each plant has several stems, leaves, and flowers, and for each plant let the following hold:

S = the average stem length (in inches)

L = the average leaf width (in inches)

F = the number of flowers

Four particular plans are examined, and the information is tabulated in the following matrix, with rows representing a single plant, and columns in the order {S, L, F}:

In [1]:
import numpy as np
flowers = np.array([[1, 1, 10],[2, 1, 12], [2, 2, 15], [3, 2, 17]])
print(flowers)

[[ 1  1 10]
 [ 2  1 12]
 [ 2  2 15]
 [ 3  2 17]]


5\. (Continued) Find the SVD of the matrix. Use the outputs from SVD to determine if there exists a linear relationship between S, L, and F. In other words, does there exist constants $\alpha_0$, $\alpha_1$, $\alpha_2$, $\alpha_3$ such that $\alpha_0 + \alpha_1 S + \alpha_2 L + \alpha_3 F = 0$. 

_Hint: You may want to determine yourself, via algebra, if there exists a linear relationship, then determine what parts of SVD help explain this and how_

6\. Turn yourself (or a classmate) into a mermaid. Or at least show that you (they) might have played on on TV. I have included several images of dolphins and fish in the exam folder (though you are welcome to find another one if you want). If you really struggle with this, I have also included an image of a "mermaid photo board". However, if you use this, you will NOT receive full credit. 

In addition to the CODE (and final image), you need to include an explanation of the process, and the mathematics you used to accomplish this feat!